In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from math import ceil
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [2]:
## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
        
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [3]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
## Merging by concat to not lose dtypes
#def merge_by_concat(df1, df2, merge_on):
#    merged_gf = df1[merge_on]
#    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
#    new_columns = [col for col in list(merged_gf) if col not in merge_on]
#    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
#    return df1

In [5]:
########################### Vars
#################################################################################
TARGET = 'fact'         # Our main target
END_TRAIN = '20230108'         # Last day in train set
MAIN_INDEX = ['id','calday']  # We can identify item by these columns

In [19]:
dir_ = r'C:/Users/vadim.sokolov/Desktop/В/LGBM prediction for Cannibalization/A1 First place/' # input only here
raw_data_dir = dir_+'2. data/'
processed_data_dir = dir_+'2. data/processed/'

df = pd.read_csv(raw_data_dir + 'LGBM_test_3tk_2pdk.csv', sep = ',')
holidays = pd.read_csv(raw_data_dir + 'Holidays_2022_2023.csv', sep = ';')

df = pd.merge(df, holidays, on = ['calday'], how = 'left').fillna('na')

df['calday_time'] = df['calday'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
df['year_iso'] = pd.to_numeric(df['year_iso'])
df['day'] = df['calday_time'].dt.day.astype(np.int8)
df['dayofweek'] = df['calday_time'].dt.dayofweek.astype(np.int8)
df['month'] = df['calday_time'].dt.month.astype(np.int8)
df['rt_promoct_unit_ABC'] = df['rt_promoct_corr'].map(str) + '-' + df['unit'].map(str) + '-' + df['ABC'].map(str)

df['productlist'] = df['plant'].map(str) + '-' + df['subcat_id'].map(str) + '-' + df['rt_promoct_corr'].map(str)
productlist = df.loc[(df['calday'] >= 20230109), 'productlist']
productlist = list(productlist.unique())
df = df[df['productlist'].isin(productlist)]
df = df.drop(columns=['productlist','unit','rt_promoct_corr','ABC'])

df['id'] = df['plant'].map(str) + '-' + df['subcat_id'].map(str) + '-' + df['rt_promoct_unit_ABC'].map(str) 
df = df.sort_values(['id','calday'],ascending=True)
df = df[['year_iso', 'month', 'week_iso', 'calday', 'day', 'dayofweek', 'calday_time', 'Holiday_type', 
        'region_num', 'format', 'plant', 'group_id', 'cat_id', 'subcat_id', 'rt_promoct_unit_ABC', 'id', 
        'flag_oos',  'duration_rt_promoct_days_in_week_max', 'day_sales_qty_total', 'TotalSales', 'avg_sales_day_sku_plant', 
        'price_total', 'fact']
         ]
len(df)

#df['flag_oos'] = df['flag_oos'].apply(math.ceil)

15315

In [20]:
df.columns

Index(['year_iso', 'month', 'week_iso', 'calday', 'day', 'dayofweek',
       'calday_time', 'Holiday_type', 'region_num', 'format', 'plant',
       'group_id', 'cat_id', 'subcat_id', 'rt_promoct_unit_ABC', 'id',
       'flag_oos', 'duration_rt_promoct_days_in_week_max',
       'day_sales_qty_total', 'TotalSales', 'avg_sales_day_sku_plant',
       'price_total', 'fact'],
      dtype='object')

In [21]:
# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(df.memory_usage(index=True).sum())))

# We can free some memory by converting "strings" to categorical it will not affect merging and we will not lose any value data.
#index_columns = ['region_num','plant','group_id','cat_id','subcat_id','unit','rt_promoct_corr','format','ABC']
#for col in index_columns:
#    df[col] = df[col].astype('category')

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced df',sizeof_fmt(df.memory_usage(index=True).sum())))

df.to_pickle(processed_data_dir+'a. grid_part_1.pkl')

    Original grid_df:   2.5MiB
          Reduced df:   2.5MiB


In [22]:
df = reduce_mem_usage(df)

Mem. usage decreased to  1.21 Mb (51.5% reduction)


In [23]:
########################### Merge calendar #################################################################################
#grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
#icols = ['date','d','event_name_1','event_type_1','event_name_2','event_type_2', 'snap_CA','snap_TX','snap_WI']

# Minify data 'snap_' columns we can convert to bool or int8
#icols = ['event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI']
#for col in icols:
#    df[col] = df[col].astype('category')

# Convert to DateTime
#df['date'] = pd.to_datetime(df['date'])

# Make some features from date

#df['tm_w'] = ['date'].dt.week.astype(np.int8)
#df['tm_y'] = ['date'].dt.year
#df['tm_y'] = (['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
#df['tm_wm'] = ['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8) # 오늘 몇째주?

#df['tm_dw'] = ['date'].dt.dayofweek.astype(np.int8) 
#df['tm_w_end'] = (['tm_dw']>=5).astype(np.int8)

# Remove date
#del df['date']

In [24]:
# We need only 'id','d','sales' to make lags and rollings
df = df[['id','calday','fact']]
SHIFT_DAY = 28  # 28 if calday

# Lags with 28 day shift
start_time = time.time()
print('Create lags')

LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)]
df = df.assign(**{
        '{}_lag_{}'.format(col, l): df.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in LAG_DAYS
        for col in [TARGET]
    })

# Minify lag columns
for col in list(df):
    if 'lag' in col:
        df[col] = df[col].astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

# Rollings with 28 day shift
start_time = time.time()
print('Create rolling aggs')

# for days
for i in [7,14,30,60,180]:
    print('Rolling period:', i)
    df['rolling_mean_'+str(i)] = df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
    df['rolling_std_'+str(i)]  = df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

# Rollings with sliding shift
for d_shift in [1,7,14]: 
    print('Shifting period:', d_shift)
    for d_window in [7,14,30,60]:
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        df[col_name] = df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
    
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

df.to_pickle(processed_data_dir+'1. lags_df_'+str(SHIFT_DAY)+'.pkl')
df.to_excel(processed_data_dir+'1. lags_df_'+str(SHIFT_DAY)+'.xlsx')

Create lags
0.01 min: Lags
Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 180
Shifting period: 1
Shifting period: 7
Shifting period: 14
0.03 min: Lags


In [25]:
df = pd.read_pickle(processed_data_dir+'a. grid_part_1.pkl')
df.columns

Index(['year_iso', 'month', 'week_iso', 'calday', 'day', 'dayofweek',
       'calday_time', 'Holiday_type', 'region_num', 'format', 'plant',
       'group_id', 'cat_id', 'subcat_id', 'rt_promoct_unit_ABC', 'id',
       'flag_oos', 'duration_rt_promoct_days_in_week_max',
       'day_sales_qty_total', 'TotalSales', 'avg_sales_day_sku_plant',
       'price_total', 'fact'],
      dtype='object')

In [ ]:
COLUMNS(['year_iso', 'week_iso', 'calday', 
         
         'region_num', 'plant', 'group_id','cat_id', 'subcat_id', 
         'flag_oos', 'format', 'price_total', 'duration_rt_promoct_days_in_week_max', 'fact', 'day_sales_qty_total',
         'TotalSales', 'avg_sales_day_sku_plant', 
         'Holiday_type', 
         'calday_time',
         'day', 'dayofweek', 'month', 
         'rt_promoct_unit_ABC']

In [27]:
df = pd.read_pickle(processed_data_dir+'a. grid_part_1.pkl')
df['fact'][df['calday']>20230109] = np.nan
base_cols = list(df)

icols =  [
            ['region_num'],                  ###  ['region_num']                ['state_id'],
            ['plant'],                       ###  ['plant']                     ['store_id'],
            # ['group_id'],
            ['cat_id'],                      ###  ['cat_id']                    ['cat_id'],
            ['subcat_id'],                   ###  ['subcat_id']                 ['dept_id'],
            ['region_num', 'cat_id'],        ###  ['region_num', 'cat_id']      ['state_id', 'cat_id'],
            ['region_num', 'subcat_id'],     ###['state_id', 'dept_id'],        ['region_num', 'subcat_id']
            ['plant', 'cat_id'],             ###['store_id', 'cat_id'],         ['plant', 'cat_id']
            ['plant', 'subcat_id'],          ###['store_id', 'dept_id'],        ['plant', 'subcat_id']
            ['rt_promoct_unit_ABC'],               ###['item_id']               ['rt_promoct_unit_ABC']
            ['rt_promoct_unit_ABC', 'region_num'], ###['item_id', 'state_id'],  ['rt_promoct_unit_ABC', 'region_num']
            ['rt_promoct_unit_ABC', 'plant']       ###['item_id', 'store_id']   ['rt_promoct_unit_ABC', 'plant']
            ]

for col in icols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    df['enc'+col_name+'mean'] = df.groupby(col)['fact'].transform('mean').astype(np.float16)
    df['enc'+col_name+'std'] = df.groupby(col)['fact'].transform('std').astype(np.float16)

keep_cols = [col for col in list(df) if col not in base_cols]
df = df[['id','calday']+keep_cols]

Encoding ['region_num']
Encoding ['plant']
Encoding ['cat_id']
Encoding ['subcat_id']
Encoding ['region_num', 'cat_id']
Encoding ['region_num', 'subcat_id']
Encoding ['plant', 'cat_id']
Encoding ['plant', 'subcat_id']
Encoding ['rt_promoct_unit_ABC']
Encoding ['rt_promoct_unit_ABC', 'region_num']
Encoding ['rt_promoct_unit_ABC', 'plant']


In [28]:
print('Save Mean/Std encoding')
df.to_pickle(processed_data_dir+'b. mean_encoding_df.pkl')
df.to_excel(processed_data_dir+'2. mean_encoding_df.xlsx')

Save Mean/Std encoding
